## Skript zum Hinzufügen von Anmerkungen in ein GLTF-Modell

Dieses Skript fügt beliebige Informationen aus einer CSV-Datei in ein 3D-Modell im GLFT-Format ein. Dabei werden die Freiheiten genutzt, die die Property "extras" bietet. Siehe hierzu: https://registry.khronos.org/glTF/specs/2.0/glTF-2.0.html#reference-extras

Als Eingabe werden eine CSV-Datei und ein GLFT-Model benötigt. Die CSV darf keine Umlaute, Anführungszeichen oder geschwungenen Klammern enthalten.

**WICHTIG: Damit die Zuordnung funktioniert, muss die CSV in einer Spalte "ID" die exakte Bezeichnung der Meshes enthalten, denen die Informationen in der jweiligen Zeile besigschreiben werden sollen.**

Erstellt von Lukas Lammers
07.02.2024

In [ ]:
#Lade Daten und Module

import pandas as pd
import json

raw_data = pd.read_csv("C:\\path\\annotations.csv", sep=";")

#Esetzte Nullwerte durch den String "nd"
raw_data = raw_data.fillna("nd")

glft_file = "C:\\path\\GLTF-Model.gltf"

try:
    with open(glft_file, 'r') as f:
        data = json.load(f)
        meshes = data.get('meshes', [])
except FileNotFoundError:
        print("Die angegebene JSON-Datei wurde nicht gefunden.")

#print(meshes)
#print(raw_data)

In [ ]:
#Wandle Dataframe in Dictionary um
def dataframe_to_dict(dataframe):
    return dataframe.to_dict(orient="records")

annotation_list = dataframe_to_dict(raw_data)
print(annotation_list)

In [ ]:
#Funktion zum Finden eines Meshes
def find_mesh_by_name(json_file, target_name):
            # Durchsuche die Liste der Meshes nach dem jeweiligen target_name
            for mesh in meshes:
                if mesh.get('name') == target_name:
                    return mesh  # Rückgabe des gefundenen Mesh-Objekts
            else:
                return None


In [ ]:
#Skript

for annotaion in annotation_list:
    #Finde das Mesh mit der ID des aktuellen Annotationsobjekts
    target_name = annotaion["ID"]
    found_mesh = find_mesh_by_name(glft_file, target_name)
    if found_mesh:
        print("Mesh mit dem Namen '{}' gefunden.".format(target_name))
        #Extrahiere das jeweilge Mesh
        filtered_mesh = [mesh for mesh in meshes if mesh["name"] == target_name]
        #Speichere das jeweilge Mesh, bevor es verändert wird
        old_mesh = filtered_mesh[0]
        #Extrahiere den Datensatz aus den Anmerkungen
        filtered_data = [annotaion for annotaion in annotation_list if annotaion["ID"] == target_name]
        #Auflösen der Listen
        filtered_mesh = filtered_mesh[0]
        filtered_data = filtered_data[0]
        #Erweitere das zuvor extrahierte Mesh um die Informationen des zuvor extrahierten Datensatzes
        filtered_mesh["extras"] = filtered_data
        #Ersetze das alte Mesh-Objekt durch das neue
        for idx, item in enumerate(data):
            if item == old_mesh:
                data[idx] = filtered_mesh        
    else:
        print("Mesh mit dem Namen '{}' nicht gefunden.".format(target_name))

#Speichere die veränderten Daten in der GLTF-Datei.
with open(glft_file, 'w') as f:
    json.dump(data, f, indent=4)
    print("Die Datei wurde erfolgreich gespeichert.")